# MyHDL Legacy code migration notes

Loose notes of migration issues we've come across. Some of them are elaborated in detail in [myhdl_changes.ipynb](myhdl_changes.ipynb). This document is volatile and may contain duplicates from other documents.

The final conclusion that was made: CyriteHDL does no longer attempt to become more MyHDL compatible, as the core is based on a different paradigm. The dogma: Support less, warn more. This is necessary to provide performant support for all kinds of language targets and simulator back ends.

## Terminology

As *hardware generators* in general are considered functions that are decorated using a

* `@always`, `@always_comb`, `@always_seq`

as opposed to simulation constructs not generating hardware in particular: `@instance`

## Thumb rules

* **Hierarchical resolution and (optional) strict typing in place**
* **Everything outside a *hardware generator* is executed and 'pythonic'**:
  * Internal representation constructs allow references to logical combinations, like:
    `a = c ^ b`
  * Such expressions creates logic in the IR. It can be referenced from inside any other logic generator.
  * Everything instanced by a variable in the `@block` context can be digital logic, signal, or reference and may be resolved into
    hardware
* Hardware generators are AST-translated to IRL (Intermediate Representation Language) and then executed for translation/transpilation.
* The simulator environment is completely different and no
  compatibility is preserved.
    
    
With this in mind, you'll have to:
    
* Turn global variables determining conditional compilation/translation into function parameters (before the `*` PEP570 construct)
* Eliminate loop constructs (`for ...`) inside generators, see [Loop issues](#Loop-issues).
* Clean up boolean constructs (see [Boolean Logic operations](myhdl_changes.ipynb#Logic-operations)):
  * Make sure to use boolean logic only within conditional statements
  * Translate to binary logic operations when assigning to a signal (**MIND THE OPERATOR PRECEDENCE!**)
* Turn `sig.next[i]` into `sig[i].next`, see [Member assignment](member_assignment.ipynb)
* Clean up variable usage (see [Variable usage](#Variable-usage)):
  * First occurence of a variable assignment to a static type (within a generator context only) defines its data type
  * Avoid using variables where possible, rather reserve an auxiliary signal or use a reference (outside the generator).
    **Note**: Variables are not supported for direct synthesis
* Function interface:
  * `@block`s only allow dedicated inputs or outputs, no inout signals (except [TristateSignals](../stdlogic.ipynb))
  * Signals passed as output can not be read from
  * Strict typing for port types and generics that should resolve to HDL, see [Interfaces](../signals_interfaces.ipynb)
* Handle class derival hierarchies:
  * Classes containing signals only are resolved as in/out upon their driver state
  * Strictly interface-typed classes must be decorated, see [Classes](#Classes)
* Revisit function calls (see [Functions](myhdl_changes.ipynb#Functions)):
  * Keep in mind that undecorated functions are **called** and their logic constructs will unroll entirely, i.e. no function definition is created in the resulting HDL
  * Turn function calls into HDL `@block` wherever possible
* Arithmetics: Handle bit sizes according to translation errors/warnings. See also [Arithmetic Pitfalls](../examples/arith_pitfalls.ipynb).

## Test unit

The following block is a simple example to start from. Modifications of this base will exhibit a few pitfalls during migration. First, we import from `cyhdl`:

In [1]:
from cyhdl import *

Then we define an auxiliary for cheap unit testing using VHDL-93 dialect and GHDL for reference testing:

In [2]:
from myirl.test.common_test import Simulator
from myirl.test import ghdl, icarus

def test(uut, param = (), debug = False):
    inst = uut(*param)
    vhdl93 = targets.vhdl.VHDL93()
    s = Simulator(vhdl93)
    s.run(inst, 20, debug = True)
    f = inst.elab(targets.VHDL, elab_all = True)
    return f

In [3]:
@block
def unit():
    a = Signal(intbv(0xaa)[8:])
    a.init = True
    q = Signal(bool())

    @instance
    def stim():
        q.next = False
        if a[0] == True and a[1] == False and a[7] == False:  # True boolean evaluation
            q.next = True
            
        yield delay(1)
        assert q == False
    
        print("DONE")
        raise StopSimulation
    
    return instances()

test(unit)

 Writing 'unit' to file /tmp/unit.vhdl 
 Creating library file /tmp/module_defs.vhdl 
WORK DIR of instance [Instance unit I/F: [// ID: unit_0 ]] /tmp/myirl_unit_mdsqywlz/
==== COSIM stdout ====
DONE
/tmp/unit.vhdl:36:9:@1ns:(assertion failure): Stop Simulation
/tmp/unit:error: assertion failed
in process .unit(cyritehdl).stim
/tmp/unit:error: simulation failed

 Writing 'unit' to file /tmp/myirl_unit_mdsqywlz/unit.vhdl 
 Creating library file module_defs.vhdl 


['/tmp/myirl_unit_mdsqywlz/unit.vhdl', 'module_defs.vhdl']

**Note** GHDL using the VHDL-93 standard will throw a 'failure' upon `StopSimulation`, causing GHDL to exit with a return code other than 0. This is handled better with VHDL-08

For the VHDL93 target, a `StopSimulation` event will now terminate the simulation without error code, *unless* a message argument is passed.

### Variations

Instead of creating an auxiliary signal, we can create references to signal combinations inside the `@block` context.
However, this may create redundant code when resolving to a HDL, as the reference is a true Python IRL object.

In [4]:
from myirl import Comment

@block
def unit():
    a = Signal(intbv(0xaa)[8:])
    a.init = True
    p, q = [ Signal(bool()) for _ in range(2) ]
    
    z = (a[7] == True) & (a[6] == False) & (a[0] == True) # Reference to binary combination of boolean expressions
    
    zb = a[7] & ~a[6] & a[0]  # True binary combination
    
    zs = Signal(bool())
    
    # New wiring 'generator' construct:
    wires = [
        zs.set(zb)
    ]

    # This is NEW and not MyHDL compatible: This expression
    # creates a logic construct that is used below inside
    # the HW generation:
    expr = a[6] == False
    
    @instance
    def stim():
        q.next = False
        p.next = True
        
        yield delay(1)
        Comment("A comment")
        if a[7] == True and expr and a[0] == True:  # True boolean evaluation
            q.next = True
            
        yield delay(1)
            
        if z:  # Evaluate reference
            q.next = True
            
        yield delay(1)

        if zb: # Evaluate binary op reference
            q.next = True
            
        if zs == True: # Check signal
            q.next = True
            
        yield delay(1)
        assert q == False

        a.next = 0xa1
        yield delay(1)
        p.next = z
        yield delay(1)

        assert p == True
    
    return instances()


f = test(unit, debug = True)

 Writing 'unit' to file /tmp/unit.vhdl 
 Creating library file /tmp/module_defs.vhdl 
WORK DIR of instance [Instance unit I/F: [// ID: unit_0 ]] /tmp/myirl_unit_ht6ewpey/
 Writing 'unit' to file /tmp/myirl_unit_ht6ewpey/unit.vhdl 
 Creating library file module_defs.vhdl 


We inserted a comment on purpose, to `grep` the neighbouring code lines. We can see the `expr` being resolved into a hardware element and emitted as plain construct into the HDL result.

In [5]:
!grep -5 comment {f[0]}

    process
    begin
        q <= '0';
        p <= '1';
        wait for 1 ns;
        -- A comment
        if (((a(7) = '1') and (a(6) = '0')) and (a(0) = '1')) then
            q <= '1';
        end if;
        wait for 1 ns;
        if (((a(7) = '1') and (a(6) = '0')) and (a(0) = '1')) then


## Variable usage

The final decision was made: Variable support is only fully supported within simulation contexts. For HDL constructs, avoid variables inside `@always` processes. They will *not* be supported for direct synthesis. Future design rules are aiming at forbidding variable usage completely inside synthesizeable constructs.

Some variable support is still present for VHDL output inside hardware constructs, however with issues that make such code non-portable:

* bool() types may not resolve to std_logic when mixed with signals
* A `Variable` type assigned to `False` yields a boolean type in the resulting HDL, whereas
  a `Signal(bool())` type assigned to a Python bool results in a `std_logic` output.
  
In order to eliminate variables, determine first what they are used for: For procedural calculation that does not emit to a hardware element, use a function returning a synthesizeable expression, for example a `@hdlmacro` (See [HDL macros](notebooks/myhdl_changes.ipynb#HDL-macros) using the IRL style `yield` notation.

### Configuration variables

Make sure to put variables that are supposed to be generic parameters past the `*` and ensure they are given either a default (when desired in the HDL output) or a type hint:

In [6]:
@block
def unit1(a : Signal, b: Signal.Output, * , PARAM : bool = False):
    @always_comb
    def worker():
        if a == 5:
            b.next = 0
        elif PARAM:
            b.next = 1
    return instances()

@block
def tb(unit):
    a, b = (Signal(intbv()[5:]) for _ in range(2))
    uut = unit(a, b, PARAM = True)
    
    return instances()

In [7]:
f = test(tb, (unit1, ), debug = False)

 Writing 'unit1' to file /tmp/unit1.vhdl 
 Writing 'tb' to file /tmp/tb.vhdl 
 Creating library file /tmp/module_defs.vhdl 
WORK DIR of instance [Instance tb I/F: [// ID: tb_0 ]] /tmp/myirl_tb_ta4jzu0e/
==== COSIM stdout ====
../../src/ieee/v93/numeric_std-body.vhdl:1613:7:@0ms:(assertion warning): NUMERIC_STD."=": metavalue detected, returning FALSE

 Writing 'unit1' to file /tmp/myirl_tb_ta4jzu0e/unit1.vhdl 
 Writing 'tb' to file /tmp/myirl_tb_ta4jzu0e/tb.vhdl 
 Creating library file module_defs.vhdl 


In [8]:
!grep -A 5 generic {f[0]}

    generic (
        PARAM: boolean := FALSE
    );
    port (
        a : in unsigned(4 downto 0);
        b : out unsigned(4 downto 0)


### Run-time parameter variant

When `PARAM` is not separated by a `*`, it will not be inferred to HDL but resolved statically. This is used for conditional compilation, however it can also emit unreachable code that will fail coverage tests. Try to avoid.

In [9]:
@block
def unit2(a : Signal, b: Signal.Output, PARAM : bool = False):
    @always_comb
    def worker():
        if a == 5:
            b.next = 0
        elif PARAM:
            b.next = 1
    return instances()

In [10]:
f = test(tb, (unit2, ), debug = False)

 Module tb: Existing instance tb, rename to tb_1 
 Writing 'unit2' to file /tmp/unit2.vhdl 
 Writing 'tb_1' to file /tmp/tb_1.vhdl 
 Creating library file /tmp/module_defs.vhdl 
WORK DIR of instance [Instance tb_1 I/F: [// ID: tb_0 to tb]] /tmp/myirl_tb_ghh3rvk2/
==== COSIM stdout ====
../../src/ieee/v93/numeric_std-body.vhdl:1613:7:@0ms:(assertion warning): NUMERIC_STD."=": metavalue detected, returning FALSE

 Writing 'unit2' to file /tmp/myirl_tb_ghh3rvk2/unit2.vhdl 
 Writing 'tb_1' to file /tmp/myirl_tb_ghh3rvk2/tb_1.vhdl 
 Creating library file module_defs.vhdl 


Note that `PARAM` is not converted into a `generic`, and occurences in the HDL code are resolved statically.

In [11]:
!cat {f[0]}

-- File generated from source:
--     /tmp/ipykernel_37900/3063955800.py
-- (c) 2016-2022 section5.ch
-- Modifications may be lost, edit the source file instead.

library IEEE;
use IEEE.std_logic_1164.all;
use IEEE.numeric_std.all;

library work;

use work.txt_util.all;
use work.myirl_conversion.all;

entity unit2 is
    port (
        a : in unsigned(4 downto 0);
        b : out unsigned(4 downto 0)
    );
end entity unit2;

architecture myhdl_emulation of unit2 is
    -- Local type declarations
    -- Signal declarations
begin
    
worker:
    process(a)
    begin
        if (a = "00101") then
            b <= "00000";
        elsif TRUE then
            b <= "00001";
        end if;
    end process;

end architecture myhdl_emulation;



## Loop issues

MyHDL allows loops inside hardware descriptions and creates generate statements in the resulting HDL.

In [12]:
Bool = Signal.Type(bool)

@block
def unit_loop0(b : Bool, N : int = 5):
    a = [ Signal(bool()) for _ in range(N) ]
    
    @always_comb
    def worker():
        a[0].next = b
        for i in range(1, 5):
            a[i].next = ~a[i-1]
        
    return instances()

This is no longer supported, except in simulation constructs implemented inside `@instance` functions.
Migration strategy:
* Move loop to the `@block` level, use procedural instancing
* Implement using `@process` or `@genprocess` constructs in the IRL or use the `@hdlmacro` construct.

Replace by:

In [13]:
@block
def unit_loop1(b : Bool):
    a = [ Signal(bool()) for _ in range(5) ]
    
    wires = [ a[0].set(b) ]
    wires += [ a[i].set(a[i-1]) for i in range(1, 5)]       
    return instances()

or within a library, using IRL:

In [14]:
import myirl


@myirl.block
def unit_loop2(b: Bool, N = 5):
    a = [ Signal(bool()) for _ in range(N) ]
    
    @myirl.genprocess()
    def worker():
        yield [ a[0].set(b) ]
        yield [ a[i].set(~a[i-1]) for i in range(1, N) ]
            
    return instances()

Examine VHDL output:

In [15]:
def test_loop(unit):
    b = Bool()
    uut = unit(b, 8)
    f = uut.elab(targets.VHDL)
    return f
    
f = test_loop(unit_loop2)

 Writing 'unit_loop2' to file /tmp/myirl_unit_loop2_5w6fpfo5/unit_loop2.vhdl 


In [16]:
! grep -A 30 architecture {f[0]}

architecture myIRL of unit_loop2 is
    -- Local type declarations
    -- Signal declarations
    signal a0 : std_ulogic;
    signal a1 : std_ulogic;
    signal a2 : std_ulogic;
    signal a3 : std_ulogic;
    signal a4 : std_ulogic;
    signal a5 : std_ulogic;
    signal a6 : std_ulogic;
    signal a7 : std_ulogic;
begin
    
worker:
    process(b, a0, a1, a2, a3, a4, a5, a6)
    begin
        a0 <= b;
        a1 <= not a0;
        a2 <= not a1;
        a3 <= not a2;
        a4 <= not a3;
        a5 <= not a4;
        a6 <= not a5;
        a7 <= not a6;
    end process;

end architecture myIRL;



Obviously, loop statements unroll explicitely. Thus, this is not intended for iterating through a large array.
See also [Loops](../notebooks/myhdl_changes.ipynb/#Loops).

## Classes

Recommended approach for porting legacy class constructs containing signals:

* Derive a class from your existing, undecorated signal class
* Decorate this derived class, depending on desired interface resolving:
  * `@container(CONTAINER_INTERFACE)` for a bidirectional class (creates container structures)
  * `@container(CONTAINER_LEGACY)` for legacy class behaviour (resolves each member of the signal)
  * `@container(CONTAINER_BULK)` for unidirectional internal types supported by specific targets only

For bidirectional classes, you need to define input, output and auxiliary ports (always inputs).

This allows to still use your legacy class constructs from older myHDL code.

The derived classes can be used for type specification in the interface.

Further details:
* [Class signals](myhdl_changes.ipynb#Class-signals)
* [Port Signal classes and methods](../ports.ipynb)